#### Now I try to play with the radial gradient of Balmer decrement

### !!! THis is the initial attempt to plot pixel to pixel balmer decrement: very noisy and messy
### updated method available in surfacebrightness.ipynb notebook

In [1]:
from    astropy.table       import Table
import  magpie              as     map
import  numpy               as     np
from    astropy.io          import fits
from    astropy.cosmology   import Planck18
import  astropy.units       as     u
import  matplotlib.pyplot   as     plt
import  matplotlib.colors   as     colors  
import matplotlib
#matplotlib.use('Agg')
from    functools           import reduce
from    tqdm                import tqdm
import  os
import  gc                                         
import warnings

warnings.filterwarnings("ignore", message="marker is redundantly defined")

#this is just a handy little function to return the desired file path
#give one entry in the object list, return the desired file path
def file_path(obj,prefix,filetype='fits'):
    if   filetype == 'fits':
        return f"data/{obj['field']}/{obj['field']}_{str(obj['id']).zfill(5)}.{prefix}.{filetype}"
    elif filetype == 'png':
        return f"png/{obj['field']}/{obj['field']}_{str(obj['id']).zfill(5)}.{prefix}.{filetype}"

In [2]:
#calculate distance of each pixel to center:
def image_to_center_distance(obj):
    y,x = np.indices((50,50))
    center = (np.array((50,50)) - 1) / 2
    distance = np.sqrt((x - center[1])**2 + (y - center[0])**2)
    pixel_length = np.deg2rad(0.04/3600) * Planck18.angular_diameter_distance(obj['redshift']).to(u.kpc)
    return (distance*pixel_length).value


def extract_radial_profile(hdu,obj):
    seg = hdu[2].data == obj['id']
    mask = np.logical_and(hdu[4]>0,hdu[6]>0)
    distances_obj  = image_to_center_distance(obj)[mask].value
    sort = np.argsort(distances_obj)
    distances_obj = distances_obj[sort]
    balmer_obj     = hdu[13].data[mask][sort]
    balmer_obj_err = hdu[14].data[mask][sort]
    dtype = [('DISTANCE [kpc]', 'f4'), ('BALMER_DECREM', 'f4'), ('BALMER_DECREM_ERR', 'f4')]
    return fits.BinTableHDU(np.array(list(zip(distances_obj, balmer_obj, balmer_obj_err)), dtype=dtype),name='PIX_RAD_PROFILE')



In [ ]:

obj_lis = Table.read('spectra-fitting_selected_psfmatched.fits')
for obj in tqdm(obj_lis):
    update_radial_profile = False
    if not update_radial_profile: continue
    path = file_path(obj=obj,prefix='extracted')
    with fits.open(path,mode='update') as hdu:
            new_table = extract_radial_profile(hdu,obj)
            if len(hdu)>15:
                hdu[15] = new_table
                hdu.flush()
            else:
                hdu.append(new_table)
                hdu.flush

### plot halpha, convolved hbeta and radial balmer decrement map (pixel-to-pixel)


In [ ]:
%matplotlib qt5
i= 110
obj_lis = Table.read('spectra-fitting_selected_psfmatched.fits')
for obj in tqdm(obj_lis[i:i+1]):
    path = file_path(obj=obj,prefix='extracted')
    with fits.open(path) as hdu:

        seg = hdu[2].data == obj['id']

        ax = plt.figure(figsize=(10,10));i=1
        for index in [4,11]:
            ax.add_subplot(int(f'22{i}'))
            plt.imshow(np.where(seg,hdu[index].data,0),
                        norm=colors.Normalize(vmin=0),
                        origin='lower',
                        cmap = 'plasma_r')
            plt.title(hdu[index].name)
            i+=1

        ax.add_subplot(223)
        plt.imshow(np.where(seg,hdu[13].data,0),
                    norm=colors.Normalize(vmin=0,vmax=15),
                    origin='lower',
                    cmap = 'PuBuGn_r')
        plt.title(hdu[13].name)
        
        balmer_obj     = hdu[13].data[seg]   
        balmer_obj_err = hdu[14].data[seg]
        distances_obj  = image_to_center_distance(obj)[seg].value
        print(obj['redshift'])
        mask = reduce(np.logical_and,
                [distances_obj <20 , 
                balmer_obj_err.flatten()<100, 
                #balmer_obj<70000000,
                hdu[5].data[seg]  > 0,
                hdu[11].data[seg] > 0        ])
        
        ax.add_subplot(224)
        #plt.errorbar(distances_obj[mask],balmer_obj.flatten()[mask]
        #            ,yerr=balmer_obj_err.flatten()[mask],
        #            linewidth=0,marker='.',elinewidth=1)
        plt.plot(distances_obj[distances_obj.sort()],linewidth=0,marker='.')
        plt.title('radial_profile')
        plt.ylim(0,10)
        plt.show()



# multi thread

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
%matplotlib inline
def process_fits(obj):
    try:
        path = file_path(obj=obj,prefix='extracted')
        with fits.open(path) as hdu:

            seg = hdu[2].data == obj['id']

            ax = plt.figure(figsize=(10,10));i=1
            for index in [4,11]:
                ax.add_subplot(int(f'22{i}'))
                plt.imshow(np.where(seg,hdu[index].data,0),
                            norm=colors.Normalize(vmin=0),
                            origin='lower',
                            cmap = 'plasma_r')
                plt.title(hdu[index].name)
                i+=1

            ax.add_subplot(223)
            plt.imshow(np.where(seg,hdu[13].data,0),
                        norm=colors.Normalize(vmin=0,vmax=15),
                        origin='lower',
                        cmap = 'PuBuGn_r')
            plt.title(hdu[13].name)
            
            balmer_obj     = hdu[13].data[seg]   
            balmer_obj_err = hdu[14].data[seg]
            distances_obj  = image_to_center_distance(obj)[seg].value
            mask = reduce(np.logical_and,
                    [distances_obj <10 , 
                    balmer_obj_err.flatten()<100, 
                    #balmer_obj<70000000,
                    hdu[5].data[seg]  > 0,
                    hdu[11].data[seg] > 0        ])
            
            ax.add_subplot(224)
            plt.errorbar(distances_obj[mask],balmer_obj.flatten()[mask]
                        ,yerr=balmer_obj_err.flatten()[mask],
                        linewidth=0,marker='.',elinewidth=1)
            plt.title('radial_profile')
            plt.ylim(0,10)
            plt.xlabel('radius[kpc]')
            plt.title('radial profile balmer decrement, pixel to pixel')
            plt.grid()
            plt.savefig(f"pixel_to_pixel_balmer_decrem/{obj['field']}-{obj['id']}.png")
            plt.close('all')
            return f"{obj['field']}-{obj['id']}saved"
        
    except Exception as e:
        return f"! {obj['field']}-{obj['id']} failed, error{e}"

def cat_process(obj_lis,max_threads=12):
        results = []
        with ThreadPoolExecutor(max_threads) as executor:
            futures = {executor.submit(process_fits,obj):obj for obj in obj_lis}
            for future in tqdm(as_completed(futures), total=len(obj_lis), desc="Processing"):
                results.append(future.result())
        return results

if __name__ == '__main__':
    obj_lis = Table.read('spectra-fitting_selected_psfmatched.fits')
    results = cat_process(obj_lis,max_threads=10)
    number = 0
    for result in results:
        if 'error' in result:
            number +=1
            print(result)
    print('total number of obj processed:',len(results))
    print('number of failed obj',number)